### Librairies

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from scipy.spatial import cKDTree

***

### Dossiers source & destination

In [ ]:
INPUT_FOLDER = "/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/CSV_files"
OUTPUT_FOLDER = "/Users/mgerenius/VScode_Workspace/Projet_01-Carte_dExploration/data/FINAL_data"
FINAL_CSV_FILE = "FINAL_REDUCED_DATA.csv"

***

### Fusion en 1 unique fichier CSV

Détection fichiers vides

In [ ]:
def file_is_empty(file):
    """
    Vérifie si un fichier est vide : taille < 5 octets

    param:
    - file : fichier inspecté

    sortie : 
    - True : booléen, si le fichier est bien vide
    """
    if os.path.getsize(file) < 5:
        return True

Fusion de multiples CSV en 1 unique fichier

In [ ]:
def multiple_to_one_csv(input_folder, output_folder):
    """
    Fusionne l'ensemble des fichiers CSV pour n'en former plus qu'un seul
    
    param:
    - input_folder : emplacement des multiples fichiers CSV
    
    sortie : 
    - output_folder : dossier de stockage du fichier unique
    """
    # Création du dossier de sotie si non existant
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    df_list = []    # Création d'une liste vide

    # itération de l'ensemble des fichiers
    for element in os.listdir(input_folder):
    
        file = os.path.join(input_folder, element)  # "file" = chemin d'accès complet du fichier
        
        # Vérification que le fichier inspecté n'est pas vide
        if not file_is_empty(file):

            # Stockage des données dans un dataframe, puis ajout à la liste
            df = pd.read_csv(file, encoding='utf-8')
            df_list.append(df)

    # Création et sauvegarde du CSV final
    final_file = os.path.join(output_folder, "FINAL_dataframe.csv") # "final_file" = chemin d'accès complet du fichier final
    pd.concat(df_list).to_csv(final_file)

    return final_file

Normalisation du fichier CSV au format souhaité

In [ ]:
def dataframe_cleaning(final_csv_file):
    """ 
    Formatage du CSV final sous le format :
    |   nom     |    date    |   latitude    |   longitude   |

    param :
    - fichier à normaliser
    """
    df = pd.read_csv(final_csv_file)
    
    data_list = []

    # itération sur chaque ligne pour normalisation
    for row in df.itertuples():
        data_list.append({
                "nom": row.nom,
                "date": row.date,
                "latitude": round(row.latitude, 4), # Précision à 4 décimales ≈ 11m
                "longitude": round(row.longitude, 4)
            })
        
    # Stockage dans un dataframe & enregistrement au format CSV
    new_df = pd.DataFrame(data_list)
    new_df.to_csv(final_csv_file, encoding="utf-8")  # On écrase la version du CSV non-normalisée

Appels des fonctions

In [ ]:
FINAL_DATAFRAME = multiple_to_one_csv(INPUT_FOLDER, OUTPUT_FOLDER)

dataframe_cleaning(FINAL_DATAFRAME)

print("\nFichiers CSV fusionnés et résultat nettoyé")

***

### Visualisation des données

In [ ]:
df = pd.read_csv(FINAL_DATAFRAME)
result = df.head()
nbr_of_lines = len(df)

print(result)
print(f"\nNombre de lignes : {nbr_of_lines}")

In [ ]:
df = pd.read_csv(FINAL_DATAFRAME)

ax = df.head(nbr_of_lines).plot.scatter(
        x='longitude',
        y='latitude',
        color='DarkBlue',
        title='Positions GPS'
    )

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()


***

### Réduction du nombre de données

**Brouillons :**

Brouillon 1

In [ ]:
df1 = pd.read_csv(FINAL_DATAFRAME)


# print(df.head(5))
# print(len(df1))


# for i in range(5):
#     print(df1.iloc[i])


df1 = df1.drop(index=2).reset_index(drop=True)  # Suppression de ligne
df1 = df1.iloc[:10].reset_index(drop=True)  # Troncage du dataframe
df1 = df1.drop(columns='Unnamed: 0')    # suppresion d'une colonne

lat_ligne_idx_0 = df1.loc[4]['latitude']    # latitude de la 5e ligne


print(lat_ligne_idx_0)
print("\n")
print(df1)

# df1.to_csv(OUTPUT_FOLDER+"test.csv", encoding="utf-8")    # Enregistrement en CSV

Brouillon 2

In [ ]:

# a : Biogroup
a_lat = 48.885132
a_long = 2.172250

# b : Rueil Ma vision
b_lat = 48.885393
b_long = 2.172434

# c : Rueil Bodyhit
c_lat = 48.885798
c_long = 2.171344

a = np.array((a_lat, a_long))
b = np.array((b_lat, b_long))
c = np.array((c_lat, c_long))


euclid_dist = np.linalg.norm(a-b)
euclid_dist_2 = np.linalg.norm(a-c)

print(f"Distance entre Biogroup et Rueil ma vision : {round(euclid_dist, 5)}  = 10m")
print(f"Distance entre Biogroup et Bodyhit : {round(euclid_dist_2, 5)}    = 100m")


import math
dist = math.sqrt((a_lat-b_lat)**2 + (a_long-b_long)**2)
print(f"Vérification que la fonction np.linalg retourne bien la distance euclidienne : {dist - euclid_dist}")

**RÉSULTAT :**
Le **seuil (lambda)** à été défini à **11m** d'écart, soit une **distance euclidienne de 0.00035** entre 2 points de coordonnées

**Fonctions :**

Calcul de distance enclidienne entre 2 points
#INUTILE

In [ ]:
def euclidian_dist(a, b):
    """
    Calacule la distance euclidienne entre 2 couples de points GPS (2-dimensionnel)
    
    param : 
    - a (array-like) : couple de réels (a_lat, a_long)
    - b (array-like) : couple de réels (b_lat, b_long)

    sortie : 
    - dist (float) : distance euclidienne entre les 2 pts
    """
    dist =  float(np.linalg.norm(np.array(a) - np.array(b)))
    return round(dist, 5)

**V1 :** Algorithme de recherche de proches voisins et de réduction de données

In [ ]:
# #def data_reduction_algo(final_dataframe_csv):
# """ 
# Réduit la quantité de données GPS présente dans le fichier CSV final.
# Passe en revue chaque couple de coordonnées (latitude, longitude) et supprime du dataframe 
# tous les autres couples ayant une distance euclidienne "très proche" du couple de référence.
# Un seuil arbitraire Lambda est fixé pour définir comme "très proches" deux couples de coordonnées.
# """


# # Définition du seuil de proximité
# Lambda = 0.00035    # ≈ 11m
# counter_suppr = 0
# counter_ok = 0

# # Création du dataframe
# df = pd.read_csv(FINAL_DATAFRAME)       # Lecture du fichier CSV
# df = df.drop(columns='Unnamed: 0')      # suppresion d'une colonne inutile

# # Point A : Itération n des N lignes du df
# for row_n in df.itertuples():
#     a_lat = row_n.latitude          # latitude de la ligne n
#     a_long = row_n.longitude        # longitude de la ligne n
#     n_1 = row_n.Index + 1           # indice de la ligne n+1, pour initialiser l'itération sur k
    
#     print(f"Taille du df : {len(df)}")
#     print(f" n = {row_n.Index}")
    
#     # Point B : Itération k des K lignes restantes (K = N - n) du df, en partant de l'indice n_1 (= n+1)
#     for row_k in df.iloc[n_1:].itertuples():
#         b_lat = row_k.latitude          # latitude de la ligne k
#         b_long = row_k.longitude        # longitude de la ligne k
        
#         # Calcul de la distance euclidienne entre le point A et le point B
#         a = np.array((a_lat, a_long))
#         b = np.array((b_lat, b_long))
#         dist = euclidian_dist(a, b)

#         # Vérification si la distance entre A et B est inférieur à Lambda, le seuil de proximité définit
#         # Si inférieure, suppression du point de coordonnées B
#         if dist < Lambda:
#             df = df.drop(index=row_k.Index).reset_index(drop=True)  # Suppression de ligne
#             counter_suppr += 1
#         else:
#             counter_ok += 1

#     print(f"lignes supprimées : {counter_suppr}")
#     print(f"lignes conservées : {counter_ok}")


#     print("\nPOINT SUIVANT\n")


**V2 :** Algorithme optimisé de recherche de proches voisins et de réduction de données

In [ ]:
def data_reduc_algo(final_csv_file):
    """ 
    Réduit la quantité de données GPS présente dans le fichier CSV final.
    Passe en revue chaque couple de coordonnées (latitude, longitude) et supprime du dataframe 
    les couples voisins ayant une distance euclidienne "très proche" du couple de référence.
    Un seuil arbitraire Lambda est fixé pour définir comme "très proches" deux couples de coordonnées.

    param:
    - 

    sortie :
    -
    """

    # Seuil de proximité
    Lambda = 0.00035        # ≈ 11m

    # Création du dataframe
    df = pd.read_csv(final_csv_file)        # Lecture du fichier CSV à réduire
    df = df.drop(columns='Unnamed: 0', errors='ignore')      # suppresion d'une colonne inutile
    

    # Suppression des lignes contenant des NaN
    df = df[['latitude', 'longitude']].dropna()
    df = df.reset_index(drop=True)
    N = len(df)

    # Extraction des coordonnées GPS et convertion du df en un tableau NumPy 2D (N, 2)
    coords = df[['latitude', 'longitude']].values   

    # Création d'un arbre 2-dimensionnel cKDTree, pour recherche spatiale
    tree = cKDTree(coords)

    # Initialisation de l'ensemble contenant les points à supprimer
    coords_to_delete = set()

    # Itération de l'ensemble des points de coords
    for i in range(N):
        # Si le pt est déjà marqué comme à supprimer, on passe
        if i in coords_to_delete:
            continue 
        
        # Recherche des proches voisins présent sous le seuil Lambda
        neighbors = tree.query_ball_point(coords[i], Lambda)    # Calcul de la distance euclidienne intégré
        neighbors = [n for n in neighbors if n != i]            # Exclusion du couple de coords d'indice i (car distance entre un point et lui même forcément = 0 donc < Lambda)
        coords_to_delete.update(neighbors)                      # Ajout des proches voisins dans l'ensemble des points à suppr 

    # Suppression finale de l'ensemble des coordonnées définies comme "proches voisins" d'un autre couple
    df_reduced = df.drop(index=coords_to_delete).reset_index(drop=True)

    print("Pour Lambda ≈ 11m")
    print(f"Lignes supprimées : {len(coords_to_delete)}")
    print(f"Lignes conservées : {len(df_reduced)}")

    return df_reduced


df_reduced = data_reduc_algo(FINAL_DATAFRAME)

**V3 :** VERSION FINALE 

In [ ]:
def data_reduc_algo(final_csv_file):
    """ 
    Réduit la quantité de données GPS présente dans le fichier CSV final.
    Passe en revue chaque couple de coordonnées (latitude, longitude) et supprime du dataframe 
    les couples voisins ayant une distance euclidienne "très proche" du couple de référence.
    Un seuil arbitraire Lambda est fixé pour définir comme "très proches" deux couples de coordonnées.

    param:
    - 

    sortie :
    -
    """

    # Seuil de proximité
    Lambda = 0.00035        # ≈ 11m

    # Création du dataframe
    df = pd.read_csv(final_csv_file)        # Lecture du fichier CSV à réduire
    df = df.drop(columns='Unnamed: 0', errors='ignore')      # suppresion d'une colonne inutile
    

    # Suppression des lignes contenant des NaN
    df = df.dropna(subset=['latitude', 'longitude'])
    df = df.reset_index(drop=True)
    N = len(df)

    # Extraction des coordonnées GPS et convertion du df en un tableau NumPy 2D (N, 2)
    coords = df[['latitude', 'longitude']].values   

    # Création d'un arbre 2-dimensionnel cKDTree, pour recherche spatiale
    tree = cKDTree(coords)

    # Initialisation de l'ensemble contenant les points à supprimer
    coords_to_delete = set()

    # Itération de l'ensemble des points de coords
    for i in range(N):
        # Si le pt est déjà marqué comme à supprimer, on passe
        if i in coords_to_delete:
            continue 
        
        # Recherche des proches voisins présent sous le seuil Lambda
        neighbors = tree.query_ball_point(coords[i], Lambda)    # Calcul de la distance euclidienne intégré
        neighbors = [n for n in neighbors if n != i]            # Exclusion du couple de coords d'indice i (car distance entre un point et lui même forcément = 0 donc < Lambda)
        coords_to_delete.update(neighbors)                      # Ajout des proches voisins dans l'ensemble des points à suppr 

    # Suppression finale de l'ensemble des coordonnées définies comme "proches voisins" d'un autre couple
    df_reduced = df.drop(index=coords_to_delete).reset_index(drop=True)

    print("Pour Lambda ≈ 11m")
    print(f"Lignes supprimées : {len(coords_to_delete)}")
    print(f"Lignes conservées : {len(df_reduced)}")
    print(df_reduced)
    

    return df_reduced
    



df_reduced = data_reduc_algo(FINAL_DATAFRAME)

***

### Enregistrement du nouveau fichier CSV

In [ ]:
df_reduced.to_csv(OUTPUT_FOLDER + "/" + FINAL_CSV_FILE, index=False, encoding="utf-8")